# Exploratory Data Analysis (EDA)
## Insurance Risk Analytics

This notebook covers the initial exploration of the insurance dataset. 
**Goals:**
- Understand data structure and quality.
- Analyze Loss Ratios (Claims/Premium).
- Identify temporal trends.
- Detect outliers.
- Explore geographical and vehicle-specific patterns.

In [ ]:
import sys
import os
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

# Add src to path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))
from src.loader import load_data
from src.cleaning import clean_data

# Set plot aesthetics
sns.set_theme(style="whitegrid")

### 1. Data Loading & Cleaning

In [ ]:
# Load Data
data_path = 'data/MachineLearningRating_v3.txt'
df = load_data(data_path)

if df is not None:
    df = clean_data(df)
    print(f"Data Shape: {df.shape}")
    
    # Feature Engineering: Loss Ratio
    # Avoid division by zero
    df['LossRatio'] = df['TotalClaims'] / df['TotalPremium'].replace(0, np.nan)
    print("Added LossRatio column.")

### 2. Data Summarization & Structure

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
# Check missing values
missing = df.isnull().sum()
missing[missing > 0]

### 3. Univariate Analysis & Outlier Detection

In [ ]:
# Histograms for Key Variables
fig, axes = plt.subplots(1, 2, figsize=(15, 6))

# Total Premium (Filtered for view)
sns.histplot(df[df['TotalPremium'] < 5000]['TotalPremium'], bins=50, ax=axes[0], color='skyblue')
axes[0].set_title('Distribution of Total Premium (< 5000)')

# Total Claims (Filtered for positive claims view)
sns.histplot(df[(df['TotalClaims'] > 0) & (df['TotalClaims'] < 50000)]['TotalClaims'], bins=50, ax=axes[1], color='salmon')
axes[1].set_title('Distribution of Total Claims (Positive & < 50k)')

plt.tight_layout()
plt.show()

In [ ]:
# Box Plots for Outlier Detection
fig, axes = plt.subplots(1, 3, figsize=(18, 6))

sns.boxplot(y=df['TotalPremium'], ax=axes[0], color='lightblue')
axes[0].set_title('Box Plot: Total Premium')

sns.boxplot(y=df['TotalClaims'], ax=axes[1], color='lightcoral')
axes[1].set_title('Box Plot: Total Claims')

if 'CustomValueEstimate' in df.columns:
    sns.boxplot(y=df['CustomValueEstimate'], ax=axes[2], color='lightgreen')
    axes[2].set_title('Box Plot: Custom Value Estimate')

plt.tight_layout()
plt.show()

### 4. Loss Ratio Analysis
Guiding Question: How does Loss Ratio vary by Province, VehicleType, and Gender?

In [ ]:
def plot_mean_loss_ratio(group_col, df, title):
    plt.figure(figsize=(10, 6))
    if group_col in df.columns:
        # Group and calculate mean loss ratio
        grouped = df.groupby(group_col)['LossRatio'].mean().sort_values(ascending=False).reset_index()
        # Filter out NaN or infinite ratios if any
        grouped = grouped[np.isfinite(grouped['LossRatio'])]
        
        sns.barplot(data=grouped, x='LossRatio', y=group_col, palette='magma')
        plt.title(title)
        plt.xlabel('Average Loss Ratio')
        plt.show()

# 1. By Province
plot_mean_loss_ratio('Province', df, 'Average Loss Ratio by Province')

# 2. By VehicleType
plot_mean_loss_ratio('VehicleType', df, 'Average Loss Ratio by Vehicle Type')

# 3. By Gender
plot_mean_loss_ratio('Gender', df, 'Average Loss Ratio by Gender')

### 5. Temporal Trends
Guiding Question: Did claim frequency or severity change over the 18-month period?

In [ ]:
# Ensure TransactionMonth is datetime
if 'TransactionMonth' in df.columns:
    # Sort by date
    time_df = df.sort_values('TransactionMonth')
    
    # Aggregate by month
    monthly_stats = time_df.groupby('TransactionMonth').agg(
        TotalPremium=('TotalPremium', 'sum'),
        TotalClaims=('TotalClaims', 'sum'),
        ClaimCount=('TotalClaims', 'count') # Or count non-zero claims if preferred
    ).reset_index()
    
    plt.figure(figsize=(14, 7))
    sns.lineplot(data=monthly_stats, x='TransactionMonth', y='TotalPremium', label='Total Premium', marker='o')
    sns.lineplot(data=monthly_stats, x='TransactionMonth', y='TotalClaims', label='Total Claims', marker='o')
    plt.title('Monthly Trends: Premium vs Claims')
    plt.ylabel('Amount')
    plt.legend()
    plt.show()

### 6. Correlation & Geography
Guiding Question: Explore relationships (Premium vs Claims) by ZipCode/Geography.

In [ ]:
# Correlation Matrix of Financial Variables
financial_cols = ['TotalPremium', 'TotalClaims', 'SumInsured', 'CalculatedPremiumPerTerm', 'LossRatio']
corr = df[financial_cols].corr()

plt.figure(figsize=(8, 6))
sns.heatmap(corr, annot=True, cmap='coolwarm', center=0)
plt.title('Correlation Matrix of Financial Variables')
plt.show()

In [ ]:
# Premium vs Claims by PostalCode (ZipCode)
# Since there are many zipcodes, let's look at the top 20 by volume
if 'PostalCode' in df.columns:
    top_zips = df['PostalCode'].value_counts().nlargest(20).index
    zip_df = df[df['PostalCode'].isin(top_zips)]
    
    plt.figure(figsize=(12, 8))
    sns.scatterplot(data=zip_df, x='TotalPremium', y='TotalClaims', hue='PostalCode', palette='tab20', alpha=0.5)
    plt.title('Premium vs Claims for Top 20 Postal Codes')
    plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', title='PostalCode')
    plt.tight_layout()
    plt.show()